In [ ]:
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import pandas as pd

# Wizualizacja grafów

## Podstawy
Do rysowania wykorzystuje się funkcje `nx.draw()`, która wywołuje funkcję `nx.draw_networkx()`. Z kolei `nx.draw_networkx()` wywołuje funkcje rysujące kolejno różne elementy grafu:
- `nx.draw_networkx_nodes()` - wierzchołki,
- `nx.draw_networkx_edges()` - krawędzie,
- `nx.draw_networkx_labels()` - etykiety wierzchołków,
- `nx.draw_networkx_edge_labels()` - etykiety krawędzi.

Można też wywołać te funkcje bezpośrednio.

In [ ]:
edgelist_df = pd.DataFrame({
    "source": ["A", "A", "A", "B", "B", "C", "D", "E"],
    "target": ["B", "C", "D", "E", "F", "G", "H", "I"],
    "weight": [5, 2, 1, 2, 3, 5, 4, 1]
})
edgelist_df

In [ ]:
nodes_values = dict(zip(["A", "B", "C", "D", "E", "F", "G", "H", "I"], [1, 2, 3, 4, 5, 6, 7, 8, 9]))
nodes_values

In [ ]:
graph = nx.from_pandas_edgelist(edgelist_df, edge_attr=True)  # tworzenie grafu z listy krawędzi (są też inne metody)
nx.set_node_attributes(graph, nodes_values, "value")  # aktualizacja atrybutów wierzchołków

In [ ]:
graph.nodes(data=True)

In [ ]:
graph.edges(data=True)

In [ ]:
plt.figure(figsize=(8, 4))
nx.draw(graph)

In [ ]:
plt.figure(figsize=(8, 4))
nx.draw(graph, with_labels=True, node_color="red", edge_color="gray")

In [ ]:
plt.figure(figsize=(8, 4))
nx.draw(
    graph,
    with_labels=True,
    node_color="red",
    node_shape="s",
    edge_color="gray",
    node_size=[v[1]["value"] * 100 for v in graph.nodes(data=True)],
    width=[e[2]["weight"] for e in graph.edges(data=True)],
)

## Pozycje wierzchołków - layout grafu 
Jest wiele sposobów na wyznaczenie pozycji wierzchołków grafu. W `networkx` dostępne są między innymi następujące metody:
- `nx.circular_layout()` - wierzchołki są rozmieszczone na okręgu,
- `nx.random_layout()` - wierzchołki są rozmieszczone losowo,
- `nx.spring_layout()` - wierzchołki są rozmieszczone zgodnie z algorytmem Fruchtermana-Reingolda (opartym na symulacji fizycznej) [**domyślny**], 
- `nx.spectral_layout()` - wierzchołki są rozmieszczone zgodnie z algorytmem bazującym na wektorach własnych Laplasjanu grafu (*graph Laplacian*),
- `nx.kamada_kawai_layout()` - wierzchołki są rozmieszczone zgodnie z algorytmem Kamady-Kawai (opartym na symulacji fizycznej).

In [ ]:
g = nx.karate_club_graph() # trochę większy graf
nx.draw(g, with_labels=True)

In [ ]:
pos = nx.spring_layout(g, seed=42) # pozycje wierzchołków - losowe, ale ustalone ziarno
pos

In [ ]:
nx.draw(g, pos=pos, with_labels=True)

In [ ]:
positions = {
    "spring": nx.spring_layout(g, seed=42, ),
    "circular": nx.circular_layout(g),
    "random": nx.random_layout(g, seed=42),
    "spectral": nx.spectral_layout(g),
    "kamada_kawai": nx.kamada_kawai_layout(g),
}

fig, axs = plt.subplots(2, 3, figsize=(16, 8))
axs = axs.flatten()
for i, (name, pos) in enumerate(positions.items()):
    axs[i].set_title(name)
    nx.draw(g, pos=pos, with_labels=True, node_size=200, font_size=8, ax=axs[i])

axs[5].set_visible(False)  # żeby nie było pustego wykresu

## Tworzenie legendy 
Niestety legendy nie da się zrobić w łatwy sposób.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
nx.draw(
    graph,
    pos=nx.spring_layout(graph, seed=42),
    with_labels=True,
    node_color="red",
    node_shape="s",
    edge_color="gray",
    node_size=[v[1]["value"] * 100 for v in graph.nodes(data=True)],
    width=[e[2]["weight"] for e in graph.edges(data=True)],
    ax=ax,
)

# dodawanie kilku legend - hacki ze StackOverflow
node_sizes = [v[1]["value"] for v in graph.nodes(data=True)]
for v in np.quantile(node_sizes, np.linspace(0, 1, 5)):
    ax.plot([], [], "rs", markersize=np.sqrt(v * 100), label=v)
first_legend = ax.legend(
    labelspacing=2, title="Node size - value", frameon=False, ncol=5, loc="center right", bbox_to_anchor=(1, -0.1)
)
ax.add_artist(first_legend)

edge_sizes = [e[2]["weight"] for e in graph.edges(data=True)]
ax.legend(
    handles=[mpl.lines.Line2D([], [], color="k", linewidth=e) for e in np.unique(edge_sizes)],
    labels=list(np.unique(edge_sizes)),
    loc = "center left", 
    bbox_to_anchor=(0, -0.1),
    frameon=False,
    title="Edge width - weight",
)
plt.show()

## Przykład wizualizacji większego grafu

In [ ]:
got_s1_edges = pd.read_csv("https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-edges.csv")
got_s1_edges.head()

In [ ]:
got_s1_nodes = pd.read_csv("https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/got-s1-nodes.csv")
got_s1_nodes.head()

In [ ]:
got_graph = nx.from_pandas_edgelist(got_s1_edges, "Source", "Target", edge_attr="Weight")

In [ ]:
node_labels_dict = got_s1_nodes.set_index("Id")["Label"].to_dict()
nx.relabel_nodes(got_graph, node_labels_dict, copy=False) # zmiana nazw wierzchołków na podstawie słownika 

In [ ]:
# do ustalenia szerokości krawędzi
weights = np.array([e[2]["Weight"] for e in got_graph.edges(data=True)])
max_weight = np.max(weights)

In [ ]:
plt.hist(weights, bins=30) # pomocniczy wykres do ustalenia legendy szerokości krawędzi
plt.show()

In [ ]:
# do ustalenia wielkości wierzchołków
node_degree = dict(got_graph.degree(weight = "Weight"))
node_size = np.array([node_degree[n] for n in got_graph.nodes()])

In [ ]:
pos = nx.kamada_kawai_layout(got_graph)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 16), dpi = 300)

# rysowanie warstwowo w celu lepszej kontroli (np. różny poziom przezroczystości)
ax.set_title("Game of Thrones season 1 network", fontsize=20)

nx.draw_networkx_nodes(got_graph, 
    pos=pos, 
    node_color="firebrick", 
    node_size=node_size/2, 
    ax=ax
)

nx.draw_networkx_edges(
    got_graph,
    pos=pos,
    width=np.sqrt(weights / max_weight) * 10,
    edge_color="gray",
    alpha=0.5, # półprzezroczyste krawędzie
    ax=ax,
)

for i, (node, (x, y)) in enumerate(pos.items()):
    ax.text(x, y - 0.02, node, fontsize=7 + np.log(node_size[i]), ha="center", va="center")

for w in [1, 5, 10, 20, 50, 100, 200]:
    ax.plot([], [], "gray", linewidth=np.sqrt(w) / np.sqrt(max_weight) * 10, label=w)
ax.legend(title="Number of interactions", frameon=False, bbox_to_anchor=(0, 0.1), loc = "lower left")
ax.text(0.01, 0.1, 
"""There are five interaction types. 
Character A and Character B are connected whenever:
- Character A speaks directly after Character B
- Character A speaks about Character B
- Character C speaks about Character A and Character B
- Character A and Character B are mentioned in the same stage direction
- Character A and Character B appear in a scene together""", 
    fontsize=8, 
    transform=ax.transAxes,
    verticalalignment="top" ) # określamy bezwzględne współrzędne tekstu
plt.axis("off")
plt.show()

## Inny pakiet do wizualizacji grafów - `netgraph`
https://netgraph.readthedocs.io/en/latest/index.html 

Niestety w wielu sytuacjach działa dość wolno, ale warto zobaczyć przykłady, np.: https://python-graph-gallery.com/500-network-chart-with-edge-bundling/. 

# `matplotlib` - poziom zaawansowany

## Konfiguracja `matplotlib`

**Jak dostosować właściwości i domyślny wygląd wykresów?**
- ustawiając `rcParams` (*runtime configuration parameters*),
- używając predefiniowanych stylów,
- tworząc/modyfikując plik `matplotlibrc` (*matplotlib runtime configuration*).

**Po co to robić?** 

- dla spójnego i jednolitego wyglądu wykresów:
    - w projekcie (raport, prezentacja, praca dyplomowa, ...),
    - w firmie (np. z wykorzystaniem szablonów, w zgodzie z identyfikacją wizualną);
- dla skrócenia czasu tworzenia wykresów i kodu potrzebnego do ich modyfikacji. 


### edytowanie `rcParams`

In [ ]:
mpl.rcParams # słownik z aktualnymi ustawieniami

In [ ]:
# domyślny wykres
np.random.seed(0)
data1 = np.random.normal(loc=-1, size=50)
data2 = np.random.normal(loc=1, size=50)

def example_plot():
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(data1)
    axs[0].plot(data2)
    axs[1].hist(data1, alpha=0.5)
    axs[1].hist(data2, alpha=0.5)
    plt.show()

example_plot()

#### zmiana domyślnego rozmiaru wykresu i czcionki
(często przydatne do raportu lub prezentacji)

In [ ]:
print(mpl.rcParams["figure.figsize"])
print(mpl.rcParams["font.size"])

In [ ]:
mpl.rcParams["figure.figsize"] = (12, 6)
mpl.rcParams["font.size"] = 15

In [ ]:
example_plot()

#### zmiana sekwencji kolorów 

In [ ]:
mpl.rcParams["axes.prop_cycle"] # domyślny cykl kolorów

In [ ]:
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(color=["darkviolet", "forestgreen"]) # interesująca nas paleta kolorów

In [ ]:
# wykres z nowymi ustawieniami
example_plot()

#### zmiana stylu konkretnej grupy elementów 
np. linii, markerów, siatki, ...

In [ ]:
# linie
mpl.rcParams["lines.linewidth"] = 4
mpl.rcParams["lines.linestyle"] = "--"

# biny histogramu
mpl.rcParams["hist.bins"] = 6

# słupki (patch)
mpl.rcParams["patch.force_edgecolor"] = True  

# siatka na wykresie
mpl.rcParams["axes.grid"] = True  # żeby była siatka na wykresie
mpl.rcParams["axes.axisbelow"] = True  # żeby siatka była pod wykresem (domyślnie jest tylko pod liniami, ale nie pod punktami/słupkami)

In [ ]:
# wykres z nowymi ustawieniami
example_plot()

In [ ]:
# parametry w obrębie jednej grupy można też zmieniać tak:
mpl.rc("lines", linewidth=1, linestyle="-.")

In [ ]:
example_plot()

#### powrót do domyślnych ustawień

In [ ]:
plt.rcdefaults()

In [ ]:
example_plot()

### gotowe style
https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html

In [ ]:
print(plt.style.available)

In [ ]:
plt.style.use("ggplot")

In [ ]:
example_plot()

In [ ]:
plt.style.use("dark_background")

In [ ]:
example_plot()

In [ ]:
plt.rcdefaults()

## `matplotlib` - przydatne wskazówki

![](https://matplotlib.org/cheatsheets/_images/cheatsheets-1.png)

### złożone, dwuwymiarowe `subplots`
- spłaszczenie `axes` - `ax.flatten()`
- `plt.tight_layout()` w przypadku problemów z nachodzeniem się elementów
- `ax.set_facecolor()` w celu wyróżnienia jednego z wykresów

In [ ]:
mu = np.linspace(-4, 4, 9)
mu 

In [ ]:
# przed
fig, axs = plt.subplots(3, 3, figsize=(8, 6))
for i in range(3):
    for j in range(3):
        index = 3*i + j # :( 
        axs[i, j].hist(np.random.normal(loc=mu[index], size=1000), bins=30)
        axs[i, j].set_xlim(-7, 7)
        axs[i, j].set_title(f"mu = {mu[index]}")
plt.suptitle("Histogramy próbki z rozkładu normalnego o różnych średnich")
plt.show()

In [ ]:
# po
fig, axs = plt.subplots(3, 3, figsize=(8, 6))
axs = axs.flatten()  # żeby nie trzeba było odwoływać się do dwuwymiarowej tablicy
for i in range(len(axs)):
    axs[i].hist(np.random.normal(loc=mu[i], size=1000), bins=30)
    axs[i].set_xlim(-7, 7)
    axs[i].set_title(f"$\mu$ = {mu[i]}") # użycie TeXa
axs[4].set_facecolor("mistyrose")  # zmiana koloru tła
plt.suptitle("Histogramy próbki z rozkładu normalnego o różnych średnich")
plt.tight_layout()  # żeby się nie nakładały
plt.show()

### dodawanie anotacji, edycja podpisów osi, ... 

In [ ]:
x = np.linspace(0, 12 * np.pi, 1000)
y = np.sin(x)

In [ ]:
# podstawowy wykres
fig, ax = plt.subplots(1, 1, figsize=(8, 2))
ax.plot(x, y, color="darkblue")
plt.show()

In [ ]:
# poprawiony wykres
fig, ax = plt.subplots(1, 1, figsize=(8, 2))
ax.plot(x, y, color="darkblue")

# zacieniowanie obszaru, dodanie anotacji
ax.fill_betweenx([-1.1, 1.1], 0, 2 * np.pi, alpha=0.5, color="lightblue")
ax.text(np.pi / 2, -0.5, "okres", ha="center", va="center", color="darkblue")

# usunięcie białych marginesów przez zmianę granic osi
ax.set_xlim(0, 12 * np.pi)
ax.set_ylim(-1.1, 1.1)

# poprawa podpisów na osi x
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(base=np.pi))  # ustawienie podziałki co pi
ax.xaxis.set_major_formatter(
    mpl.ticker.FuncFormatter(lambda val, pos: f"${val / np.pi:.0f}\pi$")
)  # użycie TeXa do wyświetlania symbolu pi
# matplotlib.ticker.FuncFormatter -- funkcja powinna przyjmować wartość i pozycję, a zwracać stringa do wyświetlenia
ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(n=2))  # podziałka co pi/2

plt.show()

### zapisywanie wykresów do pliku

**Przydatne argumenty:**
- `dpi` - dots per inch (rozdzielczość)
- `bbox_inches` - określa obszar, który ma być zapisany do pliku
- `transparent` - czy tło ma być przezroczyste


**O formatach:**
- Do raportów (w szczególności robionych w LaTeX-u) najlepiej zapisywać wykresy jako grafikę wektorową (np. `pdf` lub `svg`). Można wtedy wykorzystać np. Inkscape do dalszej edycji. 
- Jeśli konieczny jest format rastrowy, to lepiej użyć `png` niż `jpg` (lepsza jakość, wspiera przezroczystość).
- Do raportów w LaTeX-u można też użyć `pgf` (grafiki wektorowe z kodu) - wtedy wykresy są generowane przez LaTeX-a. Wymaga to zainstalowania dodatkowych pakietów, ale pozwala na zachowanie spójności czcionek i wyglądu wykresów z tekstem w dokumencie, a także na wykorzystanie na figurach LaTeX-owych poleceń (np. referencji do innego obiektu w dokumencie).

In [ ]:
fig

In [ ]:
fig.savefig("wykres1.png") 

In [ ]:
fig.savefig("wykres2.png", dpi=500, bbox_inches="tight") 

In [ ]:
fig.savefig("wykres3.png", dpi=500, bbox_inches="tight", transparent=True)

In [ ]:
fig.savefig("wykres.pdf", bbox_inches="tight") # zapis do PDFa - grafika wektorowa

In [ ]:
fig.savefig("wykres.pgf", bbox_inches="tight") # zapis do PGF